## Woody's Furniture Company
Problem by Scott Provan. Lecture Notes for OR 415: Deterministic Models in Operations Research, 2006.

- Woody’s Furniture Company produces chairs and tables.
- Chairs are made entirely out of pine and each chair uses 14 linear feet of pine.
- Tables are made of pine and mahogany and each table uses 26 linear feet of pine and 15 linear
feet of mahogany.
- Each chair requires 8 hours of labor to produce and each table requires 3 hours of labor.
- The profit from each chair is \$35 and the profit from each tables is \$60.
- Woody has 190 linear feet of pine and 60 linear feet of mahogany available each day, and has a work force of 92 labor hours each day.
- How should Woody use his resources to achieve the maximum daily profit, if the numbers of chairs and tables he makes daily have to be integers?

In [1]:
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)
#Make x1 and x2 integer
@variable(m, x[1:2] >= 0, Int)

#Constraint for pine
@constraint(m, pine, 14x[1]+26x[2] <= 190)
#Constraint for mahogany
@constraint(m, mahogany, 15x[2] <= 60)
#Constraint for labor
@constraint(m, labor, 8x[1] + 3x[2] <= 92)

#Objective function
@objective(m, Max, 35x[1]+60x[2])

print(m)

Max 35 x[1] + 60 x[2]
Subject to
 pine : 14 x[1] + 26 x[2] ≤ 190
 mahogany : 15 x[2] ≤ 60
 labor : 8 x[1] + 3 x[2] ≤ 92
 x[1] ≥ 0
 x[2] ≥ 0
 x[1] integer
 x[2] integer


In [2]:
optimize!(m)  

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2 rows, 2 cols, 4 nonzeros
2 rows, 2 cols, 4 nonzeros
Objective function is integral with scale 0.2

Solving MIP model with:
   2 rows
   2 cols (0 binary, 2 integer, 0 implied int., 0 continuous)
   4 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   625             -inf                 inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   625             410               52.44%        0      0      0         0     0.0s
 R       0       0         0   0.00%   467.65625       445                5.09%        0      0      0         2     0.0s
 T       0       0         0   0.00%   460             460                0.00%        9  

In [3]:
#Round x to deal with slight errors in late decimal points
@show round.(value.(x))
@show objective_value(m)

round.(value.(x)) = [8.0, 3.0]
objective_value(m) = 460.0


460.0

Let's try solving the LP relaxation. Only change is to not have $x$ be integer.

In [4]:
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)
#Make x1 and x2 integer
@variable(m, x[1:2] >= 0)

#Constraint for pine
@constraint(m, pine, 14x[1]+26x[2] <= 190)
#Constraint for mahogany
@constraint(m, mahogany, 15x[2] <= 60)
#Constraint for labor
@constraint(m, labor, 8x[1] + 3x[2] <= 92)

#Objective function
@objective(m, Max, 35x[1]+60x[2])

optimize!(m)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2 rows, 2 cols, 4 nonzeros
2 rows, 2 cols, 4 nonzeros
Presolve : Reductions: rows 2(-1); columns 2(-0); elements 4(-1)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.3749660398e+00 Ph1: 2(2.875); Du: 1(4.37497) 0s
          2    -4.6801204819e+02 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  4.6801204819e+02
HiGHS run time      :          0.00


In [5]:
@show value.(x)
@show round.(value.(x))
@show objective_value(m)

value.(x) = [10.975903614457831, 1.3975903614457836]
round.(value.(x)) = [11.0, 1.0]
objective_value(m) = 468.01204819277115


468.01204819277115

We get a **very** different solution by rounding the LP relaxation!